In [1]:
import pandas as pd
# from datasets import Dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    TextClassificationPipeline
    
)
import os, re
from underthesea import word_tokenize
import warnings
warnings.filterwarnings("ignore")

c:\Users\user\Documents\python\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Load danh sách công ty và mã
macp = pd.read_excel(r'C:\Users\user\Documents\python\Samsung\Data\Macp.xlsx').dropna()
tenct = macp['Tên Công ty'].tolist()
for i in range(len(tenct)):
  tenct[i] = str(tenct[i]).lower()
tenct[:5]
tenma = macp['Mã '].tolist()

# Hàm làm sạch text
def clean_text(text):
    # Thay Covid-19 thành Covid
    text = text.replace('Covid-19', 'Covid')
    
    # Thay thế tên tổ chức, người, địa điểm, mã ck... bằng 'name' / 'loc' / 'percent'
    # Ở đây ta sẽ đơn giản: thay ticker (tenma) và tên công ty (tenct)
    for ticker in tenma:
        text = re.sub(rf'\b{re.escape(ticker)}\b', 'name', text)
    for company in tenct:
        text = re.sub(rf'\b{re.escape(company)}\b', 'name', text, flags=re.IGNORECASE)
    
    # Thay số % → 'percent'
    text = re.sub(r'\d+\.?\d*%', 'percent', text)
    # Thay dates patterns → 'date'
    text = re.sub(r'\b(?:quý|Quý)\s*[1-4]\b', 'date', text)
    text = re.sub(r'\b(?:tháng|Tháng)\s*\d{1,2}\b', 'date', text)
    text = re.sub(r'\bnăm\s*\d{4}\b', 'date', text)
    text = re.sub(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b', 'date', text)
    
    # Thay số thuần túy → 'number'
    text = re.sub(r'\b\d+(\.\d+)?\b', 'number', text)
    
    # Loại bỏ ký tự đặc biệt
    text = re.sub(r'[\"“”\[\]\(\)\.,:;]', ' ', text)
    text = re.sub(r'[-–]', ' ', text)
    
    # tokenization và lowercase
    tokens = word_tokenize(text, format="text")
    return tokens.lower()


In [2]:
data = pd.read_excel(r"C:\Users\user\Documents\python\Samsung\Data\raw_data.xlsx")
data.head(5)

,text,label
0,"""Trùm"" logistics Sotrans lên kế hoạch tăng gấp...",3
1,"ĐHĐCĐ SSI: ""Khen ngợi CTCK thị phần số 1, nhưn...",2
2,Cổ phiếu nhóm Viettel giảm mạnh dù thị trường ...,2
3,"Cổ phiếu lên đỉnh, con gái chủ tịch Haxaco (HA...",2
4,"FPT: Cổ phiếu bứt phá 59%, lãi ròng 4 tháng đầ...",3


In [6]:
data['label'].value_counts()

label
3    569
2    249
1    187
Name: count, dtype: int64

In [8]:
new_title = []
title = data['text'].tolist()
for i in title:
  new_title.append(clean_text(i))
new_title


['trùm logistics sotrans lên kế_hoạch tăng gấp đôi lợi_nhuận lên number tỷ đồng',
 'đhđcđ name khen_ngợi ctck_thị_phần số number nhưng không thỏa_mãn với vị_trí số number',
 'cổ_phiếu nhóm viettel giảm mạnh dù thị_trường lên đỉnh',
 'cổ_phiếu lên đỉnh con gái chủ_tịch haxaco_name đăng_ký bán toàn_bộ cổ_phiếu',
 'name cổ_phiếu bứt_phá percent lãi_ròng number tháng đầu năm tăng percent lên mức number tỷ đồng',
 'soi lợi_nhuận date của các cổ_phiếu hàng hiệu',
 'sp mua lại percent cổ_phần skylar từ name energy xác_định việt_nam thị_trường trọng_điểm để đầu_tư dự_án năng_lượng tái_tạo',
 'name corp_name dự_kiến phát_hành gần number triệu cổ_phiếu trả cổ tức date',
 'sự vực dậy sau hàng thập_kỷ của những thương_hiệu vang bóng một thời',
 'number sai_lầm kể_cả nhà_đầu_tư chứng_khoán lâu năm cũng mắc phải',
 'chủ_tịch khẳng_định ai bỏ name sau_này ôm hận vì đâu dragon_capital vẫn mạnh_tay chốt lãi ?',
 'tiếp_tục kéo_dài thời_hạn giảm_giá dịch_vụ chứng_khoán đến hết date',
 'nghe theo khuyến_n

In [9]:
new_data = pd.DataFrame()
new_data['text'] = new_title
new_data['label'] = data['label']
new_data.head()

,text,label
0,trùm logistics sotrans lên kế_hoạch tăng gấp đ...,3
1,đhđcđ name khen_ngợi ctck_thị_phần số number n...,2
2,cổ_phiếu nhóm viettel giảm mạnh dù thị_trường ...,2
3,cổ_phiếu lên đỉnh con gái chủ_tịch haxaco_name...,2
4,name cổ_phiếu bứt_phá percent lãi_ròng number ...,3


In [12]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "vinai/phobert-base", num_labels=3
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 1) Load data
df = pd.read_excel(r"C:\Users\user\Documents\python\Samsung\Data\new_data.xlsx") 
# df = pd.read_excel(r"C:\Users\user\Documents\python\Samsung\Data\raw_data.xlsx")
df['label'] = df['label'] - 1
ds = Dataset.from_pandas(df).train_test_split(test_size=0.1)

# 2) Tokenizer & Model
model_name = "vinai/phobert-base"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
model      = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=3  # 3 lớp sentiment
)

# 3) Preprocess
def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)
ds = ds.map(preprocess, batched=True)

# 4) Metrics
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return metric.compute(predictions=preds, references=labels)

# 5) TrainingArguments
training_args = TrainingArguments(
    output_dir="phobert-sentiment-stock",
    overwrite_output_dir=True,           # nếu bạn muốn ghi đè output cũ
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    learning_rate=2e-5,
    eval_strategy="steps",               # hoặc "epoch"
    eval_steps=500,                      # đánh giá mỗi 500 bước
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,         # nếu muốn tự load best checkpoint
    metric_for_best_model="accuracy",    # dựa trên metric nào
    greater_is_better=True               # true nếu metric càng lớn càng tốt
)

# 6) Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

# 7) Fine‑tune
trainer.train()

# 8) Lưu model
trainer.save_model("phobert-sentiment-stock")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/904 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

c:\Users\user\Documents\python\DSTC_2024\round1\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


In [20]:
# Sau khi trainer.train()
metrics = trainer.evaluate()  
print("Eval metrics:", metrics)


c:\Users\user\Documents\python\DSTC_2024\round1\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Eval metrics: {'eval_loss': 0.662995457649231, 'eval_accuracy': 0.7524752475247525, 'eval_runtime': 27.9685, 'eval_samples_per_second': 3.611, 'eval_steps_per_second': 0.143, 'epoch': 3.0}


In [ ]:
test = clean_text("Tin A: tăng trưởng vượt bậc")
# Load tokenizer của PhoBERT gốc
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Load model fine-tuned
model = AutoModelForSequenceClassification.from_pretrained("phobert-sentiment-stock")

# pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, framework="pt", return_all_scores=False)

print(pipe(test)) 

Device set to use cuda:0
c:\Users\user\Documents\python\.venv\lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'LABEL_2', 'score': 0.8792685270309448}]
